# <center><font color=#76B900 size="+3"><b>**CVRPTW Exercise**</b></font></center>
---

**Learning Objectives:**
- Test your ability to model a routing optimization problem with cuOpt

### Exercise
You have the following information and constraints:
- 9 Locations:
    - 1 Depot (e.g. Office or Warehouse)
    - 8 (Clients/Customer)
- Number of vehicles 6
- Each vehicle has an associated capacities
- Each location has a specific time window within which services/deliveries need to be performed
- Demand may differ across locations
- Each vehicle has a time window of its availability
- Each vehicle has an associated priority

problem details are provided below, use NVIDIA cuOpt to determine an optimized route plan

In [ ]:
coordinates      = [ [3, 2],   [4, 8], [7, 2], [1, 3], [8, 9], [5, 1], [2, 5], [4, 2], [0, 1]]
locations        = [ "Depot",   "A",    "B",    "C",    "D",    "E",    "F",    "G",   "H"   ]
# Number of orders to be delivered at each location, this can be number of orders, weight or volumne
orders           = [   0,        3,      4,      4,      3,      4,      3,      2,     5    ]
# Earliest a delivery can be made
l_tw_earliest    = [   0,        3,      5,      3,      1,      8,      4,      0,     10   ]
# Latest a delivery can be made
l_tw_latest      = [1000,       20,     30,     15,     20,     12,     40,     30,     20   ]
# Service time required for the delivery
l_tw_service     = [   0,         1,     1,      1,       1,     2,      3,      1,     2    ]
n_locations      = len(locations)
n_vehicles       = 6
# Number of orders each vehicle can carry, this can be number of orders, weight or volumne
vehicle_capacity = [3,   8,  7,  9,  5,  10]
# Earliest a vehicle can start 
v_tw_earliest    = [0,   1,  3,  5,  20, 30]
# Latest a vehicle will be working
v_tw_latest      = [80, 40, 30, 80, 100, 40]
# Vehicle Priority, lower value higher priority
v_priorities     = [0,   3,  2,  1,  2, 100]
min_vehicles     = 4
n_climbers       = 64
time_limit       = 0.1

colors      = ["blue", "white", "green", "pink", "yellow", "orange"]


**Hints and answers have been provided below under each section.**

## Import Libraries

In [ ]:
import cuopt
import cudf
import pandas as pd
from scipy.spatial import distance
import helper_function.helper_map as helper_map

## Create DataFrame

Create a Pandas dataframe from given coordinates, this would be used to generate distance/cost matrix

In [ ]:
df = # Add your code here

df

<details><summary>Answer</summary>
<p>
df = pd.DataFrame(coordinates, columns=['xcord', 'ycord'], index=locations)
</p></details>

### Visualize Depot and Destinations

In [ ]:
# Not a mandatory, you can skip this
helper_map.gen_plot(df).show()

## Create Distance/Cost Matrix

Create a cuDF dataframe from the distance matrix

In [ ]:
#distance_matrix = # Add your code here to create distance matrix

#distance_matrix = # Add your code create cuDF Dataframe

distance_matrix

<details><summary>Answer</summary>
<p>
distance_matrix = distance.cdist(df.values, df.values, "euclidean")
    
distance_matrix = cudf.DataFrame(distance_matrix, index=df.index, columns=df.index)
</p></details>

## Create Data-Model

**Hint**: What are you planning to do with vehicle priority ? Use `set_vehicle_priority`

In [ ]:
# data_model =

<details><summary>Answer</summary>
<p>
data_model = cuopt.DataModelView(n_locations, n_vehicles)
    
data_model.set_matrix(distance_matrix)
    
data_model.add_capacity_dimension(
    "demand",
    cudf.Series(orders),
    cudf.Series(vehicle_capacity)
)
    
data_model.add_time_window(
    cudf.Series(l_tw_earliest),
    cudf.Series(l_tw_latest), 
    cudf.Series(l_tw_service)
)
    
data_model.set_vehicle_time_windows(
    cudf.Series(v_tw_earliest), 
    cudf.Series(v_tw_latest)
)
    
#### Priority for vehicles
data_model.set_vehicle_priorities(cudf.Series(v_priorities))
</p></details>

## Create Solver Instance

In [ ]:
#solver =

#routing_solution = 

<details><summary>Answer</summary>
<p>
solver = cuopt.Solver(data_model)
    
solver.set_number_of_climbers(n_climbers)
    
solver.set_time_limit(time_limit)
    
solver.set_min_vehicles(min_vehicles)
    
routing_solution = solver.solve()
</p></details>

## CVRPTW Results

In [ ]:
if routing_solution.get_status() == 0:
    print("Cost for the routing in distance: ", routing_solution.final_cost)
    print("Vehicle count to complete routing: ", routing_solution.vehicle_count)
    helper_map.show_vehicle_routes(routing_solution.route, locations)
    routing_solution.route
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", routing_solution.get_status())

In [ ]:
helper_map.map_vehicle_routes(df, routing_solution.route, colors).show()

If you can see a `Map` and see a `Final Cost` reported, this means NVIDIA cuOpt was able to find an optimized route sequence. 

From here you can modify the input data to determine its impact on result and explore other supported APIs.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)


## What's Next?

Real-world problems can potentially contain 1000s of locations and 100s of vehicles. NVIDIA cuOpt can scale to deliver unique value for problems of this size.

#### Next is an optional notebook showcasing a large CVRPTW benchmark problem. [please click to continue](./05_cvrptw_homberger_cuopt.ipynb). 

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>